# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [157]:
DATA_FOLDER = '/Users/froelich/Documents/AppliedDataAnalysis/ADA2017-Tutorials/02 - Intro to Pandas/data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [158]:
# Write your answer here

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

ANSWER:
We first import all the spreadsheets in separate dataframes.

In [189]:
import pandas as pd

mbs = []
for i in range(1,10):
    temp = pd.read_excel(DATA_FOLDER+'/microbiome/MID'+str(i)+'.xls', sheetname='Sheet 1', header=None)
    temp.columns = ['name', 'value']
    mbs.append(temp)
   
    
mbs.append(pd.read_excel(DATA_FOLDER+'/microbiome/metadata.xls', sheetname='Sheet1'))

we verify that the name column is a unique index for all the MID data and set this column as the index

In [190]:
mbs_ind = []
doublons = False
for i in range(1,10):
    mbs_ind.append(mbs[i-1].set_index(['name']))
    if mbs_ind[i-1].index.is_unique:
        mbs[i-1] = mbs_ind[i-1]
    else:
        print("Table " + i + " contains doublons")
        doublons = True

if not doublons:
    print("Tables are indexed by the column name")

Tables are indexed by the column name


we merge the MID tables

In [191]:
#pd.merge(mbs[0], mbs[1], mbs[2], how='outer', left_index=True, right_index=True)
import functools as ft
mbs_merged = ft.reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True, how='outer'), mbs[0:9])
mbs_merged.columns = ['MID1', 'MID2', 'MID3', 'MID4', 'MID5', 'MID6', 'MID7', 'MID8', 'MID9']

In [193]:
mbs_merged.head()

,MID1,MID2,MID3,MID4,MID5,MID6,MID7,MID8,MID9
name,,,,,,,,,
"Archaea ""Crenarchaeota"" Thermoprotei Acidilobales Acidilobaceae Acidilobus",NaN,2.0,1.0,NaN,5.0,NaN,NaN,NaN,NaN
"Archaea ""Crenarchaeota"" Thermoprotei Acidilobales Caldisphaeraceae Caldisphaera",NaN,14.0,15.0,NaN,26.0,NaN,1.0,NaN,1.0
"Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Ignisphaera",7.0,23.0,14.0,2.0,28.0,7.0,8.0,NaN,16.0
"Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Stetteria",NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
"Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Sulfophobococcus",NaN,1.0,4.0,NaN,5.0,1.0,2.0,NaN,2.0


Now, we use the metadata file to have meaningfull headers of our data

In [165]:
metadata = mbs[9]
metadata

,BARCODE,GROUP,SAMPLE
0,MID1,EXTRACTION CONTROL,NaN
1,MID2,NEC 1,tissue
2,MID3,Control 1,tissue
3,MID4,NEC 2,tissue
4,MID5,Control 2,tissue
5,MID6,NEC 1,stool
6,MID7,Control 1,stool
7,MID8,NEC 2,stool
8,MID9,Control 2,stool


we start be reordering the columns according to the metadata

In [194]:
mbs_order = mbs_merged[['MID1', 'MID2', 'MID6', 'MID3', 'MID7', 'MID4', 'MID8', 'MID5', 'MID9']]
mbs_order.head()

,MID1,MID2,MID6,MID3,MID7,MID4,MID8,MID5,MID9
name,,,,,,,,,
"Archaea ""Crenarchaeota"" Thermoprotei Acidilobales Acidilobaceae Acidilobus",NaN,2.0,NaN,1.0,NaN,NaN,NaN,5.0,NaN
"Archaea ""Crenarchaeota"" Thermoprotei Acidilobales Caldisphaeraceae Caldisphaera",NaN,14.0,NaN,15.0,1.0,NaN,NaN,26.0,1.0
"Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Ignisphaera",7.0,23.0,7.0,14.0,8.0,2.0,NaN,28.0,16.0
"Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Stetteria",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
"Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Sulfophobococcus",NaN,1.0,1.0,4.0,2.0,NaN,NaN,5.0,2.0


We split the table in two because the first column is 'EXTRACTION CONTROL' and do not have different types (tissue or stools)

In [195]:
mbs_part1 = mbs_order.iloc[:, :1]
mbs_part2 = mbs_order.iloc[:, 1:]
mbs_part2.head()

NEC1       control1  \
                                                   tissue stool   tissue   
name                                                                       
Archaea "Crenarchaeota" Thermoprotei Acidilobal...    2.0   NaN      1.0   
Archaea "Crenarchaeota" Thermoprotei Acidilobal...   14.0   NaN     15.0   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...   23.0   7.0     14.0   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...    NaN   NaN      NaN   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...    1.0   1.0      4.0   

                                                           NEC2        \
                                                   stool tissue stool   
name                                                                    
Archaea "Crenarchaeota" Thermoprotei Acidilobal...   NaN    NaN   NaN   
Archaea "Crenarchaeota" Thermoprotei Acidilobal...   1.0    NaN   NaN   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...   8.0    2.0   NaN   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...   NaN    NaN   NaN   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...   2.0    NaN   NaN   

                                                   control2        
                                                     tissue stool  
name                                                               
Archaea "Crenarchaeota" Thermoprotei Acidilobal...      5.0   NaN  
Archaea "Crenarchaeota" Thermoprotei Acidilobal...     26.0   1.0  
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...     28.0  16.0  
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...      1.0   NaN  
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...      5.0   2.0

We create the 2-levels index according to the metadata

In [196]:
header = pd.MultiIndex.from_product([['NEC1','control1','NEC2','control2'],
                                     ['tissue','stool']],
                                    names=['Test','Type'])
mbs_part2.columns = header

mbs_part1.columns = pd.MultiIndex.from_product([['EXTRACTION CONTROL'],['']])

Merge again all the columns together

In [197]:
mbs_final = pd.merge(mbs_part1,mbs_part2,left_index=True, right_index=True, how='outer')
mbs_final.head()

EXTRACTION CONTROL   NEC1  \
                                                                      tissue   
name                                                                           
Archaea "Crenarchaeota" Thermoprotei Acidilobal...                NaN    2.0   
Archaea "Crenarchaeota" Thermoprotei Acidilobal...                NaN   14.0   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...                7.0   23.0   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...                NaN    NaN   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...                NaN    1.0   

                                                         control1        \
                                                   stool   tissue stool   
name                                                                      
Archaea "Crenarchaeota" Thermoprotei Acidilobal...   NaN      1.0   NaN   
Archaea "Crenarchaeota" Thermoprotei Acidilobal...   NaN     15.0   1.0   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...   7.0     14.0   8.0   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...   NaN      NaN   NaN   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...   1.0      4.0   2.0   

                                                     NEC2       control2        
                                                   tissue stool   tissue stool  
name                                                                            
Archaea "Crenarchaeota" Thermoprotei Acidilobal...    NaN   NaN      5.0   NaN  
Archaea "Crenarchaeota" Thermoprotei Acidilobal...    NaN   NaN     26.0   1.0  
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...    2.0   NaN     28.0  16.0  
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...    NaN   NaN      1.0   NaN  
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...    NaN   NaN      5.0   2.0

we replace the missing values by unknown

In [198]:
mbs_final.fillna('unknown')

EXTRACTION CONTROL  \
                                                                        
name                                                                    
Archaea "Crenarchaeota" Thermoprotei Acidilobal...            unknown   
Archaea "Crenarchaeota" Thermoprotei Acidilobal...            unknown   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...                  7   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...            unknown   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...            unknown   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...            unknown   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...            unknown   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...            unknown   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...            unknown   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...                  2   
Archaea "Crenarchaeota" Thermoprotei Sulfolobal...                  3   
Archaea "Crenarchaeota" Thermoprotei Sulfolobal...            unknown   
Archaea "Crenarchaeota" Thermoprotei Thermoprot...                  3   
Archaea "Crenarchaeota" Thermoprotei Thermoprot...            unknown   
Archaea "Crenarchaeota" Thermoprotei Thermoprot...            unknown   
Archaea "Euryarchaeota" "Methanomicrobia" Metha...                  7   
Archaea "Euryarchaeota" "Methanomicrobia" Metha...            unknown   
Archaea "Euryarchaeota" "Methanomicrobia" Metha...            unknown   
Archaea "Euryarchaeota" "Methanomicrobia" Metha...            unknown   
Archaea "Euryarchaeota" "Methanomicrobia" Metha...            unknown   
Archaea "Euryarchaeota" "Methanomicrobia" Metha...                  1   
Archaea "Euryarchaeota" "Methanomicrobia" Metha...            unknown   
Archaea "Euryarchaeota" "Methanomicrobia" Metha...                  1   
Archaea "Euryarchaeota" Archaeoglobi Archaeoglo...                  1   
Archaea "Euryarchaeota" Archaeoglobi Archaeoglo...                  1   
Archaea "Euryarchaeota" Halobacteria Halobacter...            unknown   
Archaea "Euryarchaeota" Halobacteria Halobacter...            unknown   
Archaea "Euryarchaeota" Halobacteria Halobacter...            unknown   
Archaea "Euryarchaeota" Halobacteria Halobacter...                  1   
Archaea "Euryarchaeota" Halobacteria Halobacter...                  4   
...                                                               ...   
Bacteria "Proteobacteria" Gammaproteobacteria X...            unknown   
Bacteria "Proteobacteria" Gammaproteobacteria X...            unknown   
Bacteria "Proteobacteria" Gammaproteobacteria X...            unknown   
Bacteria "Proteobacteria" Gammaproteobacteria X...            unknown   
Bacteria "Proteobacteria" Gammaproteobacteria X...            unknown   
Bacteria "Proteobacteria" Gammaproteobacteria X...            unknown   
Bacteria "Spirochaetes" Spirochaetes Spirochaet...                  1   
Bacteria "Synergistetes" Synergistia Synergista...                  2   
Bacteria "Synergistetes" Synergistia Synergista...                  1   
Bacteria "Synergistetes" Synergistia Synergista...                  1   
Bacteria "Tenericutes" Mollicutes Anaeroplasmat...            unknown   
Bacteria "Tenericutes" Mollicutes Haloplasmatal...                  1   
Bacteria "Thermodesulfobacteria" Thermodesulfob...                  1   
Bacteria "Thermodesulfobacteria" Thermodesulfob...            unknown   
Bacteria "Thermotogae" Thermotogae Thermotogale...                  7   
Bacteria "Thermotogae" Thermotogae Thermotogale...                  9   
Bacteria "Verrucomicrobia" Opitutae Opitutales ...                  1   
Bacteria "Verrucomicrobia" Opitutae Opitutales ...            unknown   
Bacteria "Verrucomicrobia" Opitutae Puniceicocc...            unknown   
Bacteria "Verrucomicrobia" Opitutae Puniceicocc...            unknown   
Bacteria "Verrucomicrobia" Opitutae Puniceicocc...            unknown   
Bacteria "Verrucomicrobia" Subdivisi

safety check

In [199]:
mbs_final.index.is_unique

True

We print the result

In [201]:
mbs_final

EXTRACTION CONTROL   NEC1  \
                                                                      tissue   
name                                                                           
Archaea "Crenarchaeota" Thermoprotei Acidilobal...                NaN    2.0   
Archaea "Crenarchaeota" Thermoprotei Acidilobal...                NaN   14.0   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...                7.0   23.0   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...                NaN    NaN   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...                NaN    1.0   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...                NaN    NaN   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...                NaN    2.0   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...                NaN    1.0   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...                NaN    NaN   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...                2.0    2.0   
Archaea "Crenarchaeota" Thermoprotei Sulfolobal...                3.0   10.0   
Archaea "Crenarchaeota" Thermoprotei Sulfolobal...                NaN   11.0   
Archaea "Crenarchaeota" Thermoprotei Thermoprot...                3.0    9.0   
Archaea "Crenarchaeota" Thermoprotei Thermoprot...                NaN    5.0   
Archaea "Crenarchaeota" Thermoprotei Thermoprot...                NaN    3.0   
Archaea "Euryarchaeota" "Methanomicrobia" Metha...                7.0    9.0   
Archaea "Euryarchaeota" "Methanomicrobia" Metha...                NaN    1.0   
Archaea "Euryarchaeota" "Methanomicrobia" Metha...                NaN    NaN   
Archaea "Euryarchaeota" "Methanomicrobia" Metha...                NaN    NaN   
Archaea "Euryarchaeota" "Methanomicrobia" Metha...                NaN    NaN   
Archaea "Euryarchaeota" "Methanomicrobia" Metha...                1.0   12.0   
Archaea "Euryarchaeota" "Methanomicrobia" Metha...                NaN    NaN   
Archaea "Euryarchaeota" "Methanomicrobia" Metha...                1.0    2.0   
Archaea "Euryarchaeota" Archaeoglobi Archaeoglo...                1.0    4.0   
Archaea "Euryarchaeota" Archaeoglobi Archaeoglo...                1.0    NaN   
Archaea "Euryarchaeota" Halobacteria Halobacter...                NaN    3.0   
Archaea "Euryarchaeota" Halobacteria Halobacter...                NaN    1.0   
Archaea "Euryarchaeota" Halobacteria Halobacter...                NaN    NaN   
Archaea "Euryarchaeota" Halobacteria Halobacter...                1.0    4.0   
Archaea "Euryarchaeota" Halobacteria Halobacter...                4.0    3.0   
...                                                               ...    ...   
Bacteria "Proteobacteria" Gammaproteobacteria X...                NaN    NaN   
Bacteria "Proteobacteria" Gammaproteobacteria X...                NaN    NaN   
Bacteria "Proteobacteria" Gammaproteobacteria X...                NaN    NaN   
Bacteria "Proteobacteria" Gammaproteobacteria X...                NaN    NaN   
Bacteria "Proteobacteria" Gammaproteobacteria X...                NaN    NaN   
Bacteria "Proteobacteria" Gammaproteobacteria X...                NaN    NaN   
Bacteria "Spirochaetes" Spirochaetes Spirochaet...                1.0    NaN   
Bacteria "Synergistetes" Synergistia Synergista...                2.0    2.0   
Bacteria "Synergistetes" Synergistia Synergista...                1.0    2.0   
Bacteria "Synergistetes" Synergistia Synergista...                1.0    NaN   
Bacteria "Tenericutes" Mollicutes Anaeroplasmat...                NaN    NaN   
Bacteria "Tenericutes" Mollicutes Haloplasmatal...                1.0   15.0   
Bacteria "Thermodesulfobacteria" Thermodesulfob...                1.0    1.0   
Bacteria "Thermodesulfobacteria" Thermodesulfob...                NaN    3.0   
Bacteria "Thermotogae" Thermotogae Thermotogale...                7.0   15.0   
Bacteria "Thermotogae" Thermotogae Thermotogale...                9.0   22.0   
Bacteria "Verrucomicrobia" Opitutae Opitutales ... 

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here